<a href="https://colab.research.google.com/github/palldas/Cover-Letter-Generator/blob/main/Transformers_Cover_Letter_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAIN MENU SETUP

In [ ]:
import pandas as pd

menu_df = pd.read_csv("https://docs.google.com/spreadsheets/d/1RLipc3nXSuC7jCDRR9ajmP2Ew86f2SWmOtEORWNtE-c/export?format=csv")
menu_df

,Text,Label
0,Can you help me write a cover letter?,write_cover_letter
1,I need assistance drafting a cover letter.,write_cover_letter
2,Please help me write a cover letter.,write_cover_letter
3,I need help crafting a professional cover letter.,write_cover_letter
4,Can you guide me in writing a cover letter?,write_cover_letter
...,...,...
395,Shut this down.,exit
396,Let’s stop everything.,exit
397,End the process now.,exit
398,Stop and exit.,exit


In [ ]:
menu_df.to_csv('input.csv',index=False)

In [ ]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("csv",data_files={"train":"input.csv"},split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Text', 'Label'],
    num_rows: 400
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
from sklearn.preprocessing import LabelBinarizer

enc = LabelBinarizer()
enc.fit(menu_df['Label'])

LabelBinarizer()

In [ ]:
enc.transform(menu_df['Label'])[:10]

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [ ]:
import torch
def add_labels(example):
    example['labels'] = torch.tensor(enc.transform([example['Label']])[0]*1.0)
    return example

tokenized_datasets = tokenized_datasets.map(add_labels)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

Dataset({
    features: ['Text', 'Label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 400
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Text"])
tokenized_datasets = tokenized_datasets.remove_columns(["Label"])
#tokenized_datasets = tokenized_datasets.rename_column("Label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 400
})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model_ckpt = "bert-base-uncased"  # etc.
num_labels = 4  # etc.
# 1. Example: Help me write a cover letter
# 2. Example: Help me read a cover letter
# 3. Example: Help me complete a cover letter
# 4. Example: Exit

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=num_labels,
    problem_type="multi_label_classification",  # this is important
)

# Tokenize input text
text = "This is a great example."
inputs = tokenizer(text, return_tensors="pt")

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
logits.shape

torch.Size([1, 4])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
# here is a loop just showing how we are going to do this

# Create a DataLoader
train_dataloader = DataLoader(tokenized_datasets, batch_size=5, shuffle=True)

# Iterate through the DataLoader
for batch in train_dataloader:
    break

In [ ]:
batch

{'input_ids': tensor([[ 101, 3926, 2023,  ...,    0,    0,    0],
         [ 101, 2393, 2033,  ...,    0,    0,    0],
         [ 101, 2644, 1996,  ...,    0,    0,    0],
         [ 101, 2203, 2023,  ...,    0,    0,    0],
         [ 101, 3143, 2023,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.]])}

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/240 [00:00<?, ?it/s]

In [ ]:
text = "Please help me write the best cover letter to get a deep learning internship"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[-3.5484, -3.1743, -3.3082,  3.2706]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([3], device='cuda:0'),
 array(['write_cover_letter'], dtype='<U20'))

In [ ]:
text = "Stop this darn program."
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[-3.3313, -3.3085,  3.0778, -3.1432]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([2], device='cuda:0'),
 array(['exit'], dtype='<U20'))

In [ ]:
text = "I don't know how to finish this sentence"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[-3.1515,  2.5971, -3.8913, -3.6656]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([1], device='cuda:0'),
 array(['complete_sentence'], dtype='<U20'))

In [ ]:
text = "Can you help me read this cover letter?"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[-1.2537e+00, -3.2501e+00, -4.7955e+00,  2.5039e-03]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([3], device='cuda:0'),
 array(['write_cover_letter'], dtype='<U20'))

In [ ]:
text = "I am a recruiting manager and I need help reading a resume"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get model output
outputs = model(**inputs)

# Process output
logits = outputs.logits
predictions = torch.argmax(logits, dim=1)
logits,predictions,enc.inverse_transform(logits.cpu().detach().numpy())

(tensor([[-3.5076, -3.0462, -3.9243,  2.7993]], device='cuda:0',
        grad_fn=<AddmmBackward0>),
 tensor([3], device='cuda:0'),
 array(['write_cover_letter'], dtype='<U20'))

# Dynamic Cover Letter Generation (DONE)

---



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_gen = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model_gen = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_gen.to(device)
# print(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [ ]:
# this used causal lm generate function
# def generate_cover_letter(prompt, name):
#     separator = "<|endofprompt|>"
#     inputs = tokenizer_gen(text=prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
#     outputs = model_gen.generate(
#         inputs["input_ids"],
#         max_length=1024,
#         num_beams=4,
#         # repetition_penalty=1.2,  # Penalize repetition
#         early_stopping=True
#     )
#     ##TODO: implement generate mannually, call the forward function to predict next word

#     generated_text = tokenizer_gen.decode(outputs[0], skip_special_tokens=True)
#     if separator in generated_text:
#         generated_text = generated_text.split(separator)[-1].strip()

#     end_index = generated_text.find("Sincerely,")
#     if end_index != -1:
#         generated_text = generated_text[:end_index + len(f"Sincerely, {name}")]

#     return generated_text

In [ ]:
def generate_cover_letter(prompt, name, max_length=1024):
    separator = "<|endofprompt|>"
    inputs = tokenizer_gen(text=prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    generated_ids = inputs["input_ids"]

    for i in range(max_length):
        outputs = model_gen(generated_ids, attention_mask=inputs["attention_mask"])
        logits = outputs.logits

        next_token_logits = logits[:, -1, :]
        probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)

        next_token_id = torch.argmax(probabilities, dim=-1).unsqueeze(0)

        generated_ids = torch.cat([generated_ids, next_token_id], dim=1)

        # Stop if the special token <|endoftext|> is generated
        if next_token_id == tokenizer_gen.eos_token_id:
            break

    generated_text = tokenizer_gen.decode(generated_ids[0], skip_special_tokens=True)

    # Remove the prompt from the generated text
    if separator in generated_text:
        generated_text = generated_text.split(separator)[-1].strip()

    # Extract only up to "Sincerely," and the name
    end_index = generated_text.find("Sincerely,")
    if end_index != -1:
        generated_text = generated_text[:end_index + len(f"Sincerely, {name}")]

    return generated_text
# prompt = "Write a professional cover letter for a SWE Intern at Apple. I have entry experience in SWE Intern and am proficient in python, java, sql, pytorch, html, css, C. The company wants python, java, sql, pytorch, html, css, R so make sure the cover letter highlights how I'd be perfect for the role. Previously, swe intern at pg&e. Currently, swe intern at an entity of aws partned with my university. I also want to note that i admire apple's value of sustainability and privacy. The letter should be three paragraphs long, with each paragraph containing at least five sentences. Start with 'Dear Hiring Manager,' and end with 'Sincerely,' Pallavi Das<|endofprompt|>"
# name = "Pallavi Das"
# print(generate_cover_letter(prompt, name))

In [ ]:
# Function to generate the prompt in first person
def create_prompt(name, company_name, role, pref_qual, skill_sets, exp_level, past_exp, curr_exp, misc):
    separator = "<|endofprompt|>"
    prompt = f"Write a professional cover letter for a {role} at {company_name}. I have {exp_level} experience in {role} and am proficient in {skill_sets}. The company wants {pref_qual} so make sure the cover letter highlights how I'd be perfect for the role. Previously, {past_exp}. Currently, {curr_exp}. I also want to note that {misc}. The letter should be three paragraphs long, with each paragraph containing at least five sentences. Start with 'Dear Hiring Manager,' and end with 'Sincerely,' {name}<|endofprompt|>"
    print(prompt)
    return prompt

In [ ]:
def write_cover_letter():
    print("Let's create your personalized cover letter!")
    name = input("Enter your name: ")
    company_name = input("Enter the hiring company name: ")
    role = input("Enter the role you are applying for: ")
    pref_qual = input("Enter the company's preferred qualifications listed on the job posting: ")
    skill_sets = input("Enter your key technical & soft skills (comma-separated): ")
    exp_level = input("Enter your experience level (entry/mid/senior): ")
    past_exp = input("Enter your past working experience: ")
    curr_exp = input("Enter your current working experience: ")
    misc = input("Enter any additional information you'd like to include: ")

    prompt = create_prompt(name, company_name, role, pref_qual, skill_sets, exp_level, past_exp, curr_exp, misc)
    cover_letter = generate_cover_letter(prompt, name)

    print("\Here's the Generated Cover Letter:\n\n")
    print(cover_letter)

# Sentence Auto-Completion (done)

In [ ]:
# # Function to generate text completion
# def auto_complete_cover_letter(partial_text):
#     separator = "<|endofprompt|>"
#     prompt = f"I am writing a cover letter for my job application. Can you help me complete here is what I have so far: {partial_text}\nContinue the cover letter in a professional tone that makes me as the candidate stand out. Just continue from what I have don't change my text. Output the whole thing.<|endofprompt|>"

#     inputs = tokenizer_gen(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
#     outputs = model_gen.generate(
#         inputs["input_ids"],
#         max_length=1024,
#         num_beams=4,
#         no_repeat_ngram_size=3,  # preventing repetitive sequences
#         early_stopping=True
#     )

#     generated_text = tokenizer_gen.decode(outputs[0], skip_special_tokens=True)
#     if separator in generated_text:
#         generated_text = generated_text.split(separator)[-1].strip()

#     return generated_text

In [ ]:
def auto_complete_cover_letter(partial_text, max_length=1024):
    separator = "<|endofprompt|>"
    stop_token = "<|beginningofprompt|>"
    prompt = f"I am writing a cover letter for my job application. Can you help me complete here is what I have so far: {partial_text}\nContinue the cover letter in a professional tone that makes me as the candidate stand out:<|endofprompt|>"

    inputs = tokenizer_gen(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    generated_ids = inputs["input_ids"]

    for i in range(max_length):
        outputs = model_gen(generated_ids, attention_mask=inputs["attention_mask"])
        logits = outputs.logits

        next_token_logits = logits[:, -1, :]
        probabilities = torch.nn.functional.softmax(next_token_logits, dim=-1)

        next_token_id = torch.argmax(probabilities, dim=-1).unsqueeze(0)

        generated_ids = torch.cat([generated_ids, next_token_id], dim=1)

        if next_token_id == tokenizer_gen.eos_token_id or next_token_id.item() == tokenizer_gen.convert_tokens_to_ids(stop_token):
            break

    generated_text = tokenizer_gen.decode(generated_ids[0], skip_special_tokens=True)

    # Remove the prompt from the generated text
    if separator in generated_text:
        generated_text = generated_text.split(separator)[-1].strip()

    # Remove any occurrence of <|beginningofprompt|> and the text after it
    if stop_token in generated_text:
        generated_text = generated_text.split(stop_token)[0].strip()

    return generated_text

In [ ]:
def sentence_complete():
    print("Welcome to the Sentence Auto-Completion Tool!")
    print("Provide a part of your cover letter or sentence, and the system will complete it.\n")

    partial_text = input("Enter the beginning of your cover letter or sentence:\n")

    completed_text = auto_complete_cover_letter(partial_text)

    print("\Here's the Completed Sentence/Cover Letter:\n\n")
    print(completed_text)

# Cover Letter Analysis Classification (DONE)

In [ ]:
!pip install datasets transformers

import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import re

df = pd.read_csv("https://docs.google.com/spreadsheets/d/1nupwb1fqQvUfnduKA9cieiL11XDNp9pw3_aaJHi0aJk/export?format=csv")
display(df)

df.to_csv('cv_data.csv', index=False)

,Text,Label
0,My name is John Doe.,Applicant's Name
1,I am Jane Smith.,Applicant's Name
2,The applicant's name is Michael Johnson.,Applicant's Name
3,I am called Sarah Williams.,Applicant's Name
4,My name is David Brown.,Applicant's Name
...,...,...
497,I hope to bring enthusiasm and expertise to yo...,Other
498,I am excited to learn more about the position ...,Other
499,Thank you for the chance to be considered for ...,Other
500,I am confident in my ability to exceed your ex...,Other


In [ ]:
dataset_cv = load_dataset("csv", data_files={"train": "cv_data.csv"}, split='train')

tokenizer_cv = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer_cv(examples["Text"], padding="max_length", truncation=True)

tokenized_datasets_cv = dataset_cv.map(tokenize_function, batched=True)

enc = LabelEncoder()
enc.fit(df['Label'])
print(f"Classes: {enc.classes_}")

def add_labels(example):
    example['labels'] = torch.tensor(enc.transform([example['Label']])[0], dtype=torch.long)
    return example

tokenized_datasets_cv = tokenized_datasets_cv.map(add_labels)

tokenized_datasets_cv = tokenized_datasets_cv.remove_columns(["Text", "Label"])
tokenized_datasets_cv.set_format("torch")

train_test_split_cv = tokenized_datasets_cv.train_test_split(test_size=0.2)
train_dataset_cv = train_test_split_cv["train"]
test_dataset_cv = train_test_split_cv["test"]

train_dataloader_cv = DataLoader(train_dataset_cv, batch_size=8, shuffle=True)
test_dataloader_cv = DataLoader(test_dataset_cv, batch_size=8)

model_name = "bert-base-uncased"
model_cv = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(enc.classes_),  # Number of labels
)

optimizer = AdamW(model_cv.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader_cv)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cv.to(device)

progress_bar = tqdm(range(num_training_steps))
model_cv.train()

for epoch in range(num_epochs):
    for batch in train_dataloader_cv:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_cv(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# Sentence classification function
def classify_cover_letter(cover_letter):
    # Split the cover letter into sentences
    sentences = re.split(r'[.?!]\s*', cover_letter.strip())
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    results = {label: [] for label in enc.classes_}

    for sentence in sentences:
        inputs = tokenizer_cv(sentence, return_tensors="pt", truncation=True, padding="max_length").to(device)

        model_cv.eval()
        outputs = model_cv(**inputs)
        logits = outputs.logits
        predicted_label_id = torch.argmax(logits, dim=1).item()  # Use argmax for classification
        predicted_label = enc.classes_[predicted_label_id]

        results[predicted_label].append(sentence)

    return results

# Example cover letter
# cover_letter = """
# "Dear Hiring Manager,

# I am writing to express my interest in the Data Scientist position at XYZ Analytics. With my strong background in coding, proficiency in various BI tools, and experience in developing BI interfaces, I believe I am wellsuited for this role.

# In my current position as a Data Scientist at XYZ Analytics, I have successfully designed and executed BI dashboards that meet visual and quality standards. I have also developed technical documents to ensure transparency in the build process. Additionally, I have performed quality assurance processes to verify accuracy to specifications.

# I have a Bachelor's Degree in Computer Science and over 5 years of experience in developing BI tools/interfaces. My skillset includes coding languages such as HTML, SQL, JavaScript, and VBA, as well as proficiency in MS Office Suite and SPSS. I am also experienced in using BI tools such as Tableau, Power BI, SAP Business Objects, IBM Cognos, and Dapresy/Forsta.

# I am a selfstarter with strong organizational skills and high attention to detail. I am able to work independently or as part of a team, and I thrive in a fastpaced environment. I am confident that my qualifications and experience make me a strong candidate for this position.

# Thank you for considering my application. I look forward to the opportunity to discuss how my skills and experience align with the needs of XYZ Analytics.

# Sincerely,
# John Smith"
# """

# # Classify the cover letter
# classified_results = classify_cover_letter(cover_letter)

# # Print classified sentences by label
# print("Cover Letter Analysis:")
# for label, sentences in classified_results.items():
#     print(f"\n{label}:")
#     for sentence in sentences:
#         print(f"  - {sentence}")


Classes: ["Applicant's Name" 'Career Objective' 'Education' 'Other' 'Skills']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/153 [00:00<?, ?it/s]

In [ ]:
def analyze_cover_letter():
    # Classify the cover letter
    print("\nWelcome to the Cover Letter Analyzer:")
    cover_letter = input("Provide a cover letter to extract meaningul information:\n")
    classified_results = classify_cover_letter(cover_letter)

    for label, sentences in classified_results.items():
        print(f"\n{label}:")
        for sentence in sentences:
            print(f"  - {sentence}")

# cover_letter = """Dear Hiring Manager,

# My name is Jane Doe, and I am excited to apply for the Software Engineering Internship position at Meta. I am a third-year Computer Science student at Stanford University with a strong passion for software engineering and scalable technologies.

# I have completed courses in Data Structures, Algorithms, and Distributed Systems, which have equipped me with a solid foundation in computer science principles. At Stanford, I worked on a team project where we developed a real-time collaboration platform using React, Node.js, and Firebase, which was recognized as a finalist in the Stanford Hackathon.

# Last summer, I interned at ABC Tech, where I contributed to optimizing backend APIs for a customer analytics platform. By implementing caching strategies and database indexing, I reduced query response times by 40%, significantly improving system performance. My technical expertise includes proficiency in Python, Java, and C++, as well as experience with frameworks like React and tools such as Docker and Kubernetes.

# My ultimate career objective is to design scalable software solutions that empower global communities. I am inspired by Meta’s mission to connect people through innovative technology and its commitment to advancing the metaverse. I believe this internship will provide an excellent opportunity to further refine my skills while contributing to impactful projects.

# Thank you for considering my application. I have attached my resume, which provides additional details about my projects and experience. I look forward to the opportunity to contribute to Meta’s engineering team and would be happy to discuss my application further.

# Sincerely,
# Jane Doe
# """
# print(analyze_cover_letter())



Welcome to the Cover Letter Analyzer:
Provide a cover letter to extract meaningul information:
Dear Hiring Manager,  My name is Jane Doe, and I am excited to apply for the Software Engineering Internship position at Meta. I am a third-year Computer Science student at Stanford University with a strong passion for software engineering and scalable technologies.  I have completed courses in Data Structures, Algorithms, and Distributed Systems, which have equipped me with a solid foundation in computer science principles. At Stanford, I worked on a team project where we developed a real-time collaboration platform using React, Node.js, and Firebase, which was recognized as a finalist in the Stanford Hackathon.  Last summer, I interned at ABC Tech, where I contributed to optimizing backend APIs for a customer analytics platform. By implementing caching strategies and database indexing, I reduced query response times by 40%, significantly improving system performance. My technical expertise

# MAIN CLI

In [ ]:
def interactive_cli():
    print("Welcome to the Cover Letter AI Assistant!")
    print("You can ask me to help with tasks like writing a cover letter, analyzing one, completing a sentence, or exiting the program.")

    while True:
        user_input = input("\nHow can I assist you today?\n")

        inputs = tokenizer(text, return_tensors="pt").to(device)

        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        intent = enc.inverse_transform(logits.cpu().detach().numpy())[0]
        print(intent)

        if intent == "write_cover_letter":
            print(write_cover_letter())
        elif intent == "complete_sentence":
            print(sentence_complete()
        elif intent == "analyze_cover_letter":
            print(analyze_cover_letter())
        elif intent == "exit":
            print("\nExiting program!")
            break
        else:
            print("\nI'm not sure how to help with that. Could you rephrase or provide more details?")

In [ ]:
interactive_cli()